In [2]:
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):

    import os
    import pickle

    import pandas as pd
    from sklearn.linear_model import ElasticNet
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from sklearn.model_selection import train_test_split

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )

    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)

    estimator.fit(x_train, y_train)
    y_pred = estimator.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    if verbose > 0:
        print(estimator, ":", sep="")
        print(f"  MSE: {mse}")
        print(f"  MAE: {mae}")
        print(f"  R2: {r2}")

    if not os.path.exists("estimator.pickle"):
        saved_estimator = None
    else:
        with open("estimator.pickle", "rb") as file:
            saved_estimator = pickle.load(file)

    if saved_estimator is None or estimator.score(
        x_test, y_test
    ) > saved_estimator.score(x_test, y_test):
        with open("estimator.pickle", "wb") as file:
            pickle.dump(estimator, file)
            
            

In [3]:
train_estimator(0.5, 0.5)
            
            
#
# Experimiento
#
train_estimator(0.2, 0.2)
            

ElasticNet(alpha=0.5, random_state=12345):
  MSE: 0.5294843132862007
  MAE: 0.5894666734018875
  R2: 0.13368827268570616
ElasticNet(alpha=0.2, l1_ratio=0.2, random_state=12345):
  MSE: 0.43869119518947153
  MAE: 0.5236106762028768
  R2: 0.2822387414965034


In [4]:
def load_data():

    import pandas as pd

    url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    df = pd.read_csv(url, sep=";")

    y = df["quality"]
    x = df.copy()
    x.pop("quality")

    return x, y

In [5]:
#
# Particionamiento de datos
#
def make_train_test_split(x, y):

    from sklearn.model_selection import train_test_split

    (x_train, x_test, y_train, y_test) = train_test_split(
        x,
        y,
        test_size=0.25,
        random_state=0,
    )
    return x_train, x_test, y_train, y_test


In [6]:
#
# Cálculo de metricas de evaluación
#
def eval_metrics(y_true, y_pred):

    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mse, mae, r2
  

In [7]:
#
# Reporte de métricas de evaluación
#
def report(estimator, mse, mae, r2):

    print(estimator, ":", sep="")
    print(f"  MSE: {mse}")
    print(f"  MAE: {mae}")
    print(f"  R2: {r2}")

In [8]:
#
# Almacenamiento del modelo
#
def save_best_estimator(estimator):

    import os
    import pickle

    with open("estimator.pickle", "wb") as file:
        pickle.dump(estimator, file)

In [9]:
#
# Carga del modelo
#
def load_best_estimator():

    import os
    import pickle

    if not os.path.exists("estimator.pickle"):
        return None
    with open("estimator.pickle", "rb") as file:
        estimator = pickle.load(file)

    return estimator

In [10]:
#
# Entrenamiento
#
def train_estimator(alpha=0.5, l1_ratio=0.5, verbose=1):

    from sklearn.linear_model import ElasticNet

    x, y = load_data()
    x_train, x_test, y_train, y_test = make_train_test_split(x, y)
    estimator = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=12345)
    estimator.fit(x_train, y_train)
    mse, mae, r2 = eval_metrics(y_test, y_pred=estimator.predict(x_test))
    if verbose > 0:
        report(estimator, mse, mae, r2)

    best_estimator = load_best_estimator()
    if best_estimator is None or estimator.score(x_test, y_test) > best_estimator.score(
        x_test, y_test
    ):
        best_estimator = estimator

    save_best_estimator(best_estimator)
    